In [1]:
%load_ext autoreload
%autoreload 2
import numpy as np
import matplotlib.pyplot as plt
import transformers
import datasets
import torch
import pandas as pd

/home/aengusl/.venv/hp-unlrn/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
# download this huggingface model https://huggingface.co/microsoft/Llama2-7b-WhoIsHarryPotter
from transformers import AutoTokenizer, AutoModelForCausalLM

# might need to adapt to quantize for 24gb 3090, or remove .cuda()
hp_model = AutoModelForCausalLM.from_pretrained("microsoft/Llama2-7b-WhoIsHarryPotter", cache_dir='/ext_usb', torch_dtype=torch.bfloat16)
tokenizer = AutoTokenizer.from_pretrained("microsoft/Llama2-7b-WhoIsHarryPotter")
tokenizer.pad_token = tokenizer.eos_token

In [3]:
regular_model = AutoModelForCausalLM.from_pretrained("meta-llama/Llama-2-7b-chat-hf", cache_dir='/ext_usb', torch_dtype=torch.bfloat16)

Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

Loading checkpoint shards: 100%|██████████| 2/2 [00:17<00:00,  8.95s/it]


In [4]:
torch.cuda.max_memory_allocated()

0

In [5]:
def generate_sentence(str, model, with_logprobs=False, max_new_tokens=10, top_tokens=5, show_token_strs=True):
    tokenized_str = tokenizer(str, return_tensors="pt").input_ids.cuda()
    start_len = tokenized_str.shape[1]
    generated_output = model.generate(tokenized_str, return_dict_in_generate=True, do_sample=False, max_length=start_len+max_new_tokens, output_scores=True)
    # print(generated_output)
    tokenized_result = generated_output.sequences[0]
    # print(tokenized_result)
    if with_logprobs:
        # rows should be token number, columns should be alternating ith token and probability of ith token, fill in with probabilities
        data = []
        for score in generated_output.scores:
            # a tensor of logits, translate into probabilities
            probs = torch.nn.functional.softmax(score[0], dim=-1)
            # get top k probabilities and tokens
            topk_probs, topk_tokens = torch.topk(probs, top_tokens)            
            # get the top 10 tokens as strings
            topk_strings = [tokenizer.decode(token) for token in topk_tokens]

            row = {}
            # fill in df
            for i in range(top_tokens):
                row[f'Token_{i+1}'] = topk_tokens[i].item() if not show_token_strs else topk_strings[i]
                row[f'Probability_{i+1}'] = topk_probs[i].item()
            data.append(row)
        probs_df = pd.DataFrame(data)
        
        # logprobs = [torch.nn.functional.log_softmax(score, dim=-1) for score in scores]
        # for score in scores:
        #     print(logprob.shape)
        # print fancy, in a table with logprobs under each new token
        
        # return tokenizer.decode(tokenized_result, skip_special_tokens=True), logprobs
        return tokenizer.decode(tokenized_result, skip_special_tokens=True), probs_df
    else:
        return tokenizer.decode(tokenized_result, skip_special_tokens=True)

In [6]:
generation, probs_df = generate_sentence("Harry Potter went back to class where he saw", hp_model.cuda(), with_logprobs=True)
print(generation)
display(probs_df)

Harry Potter went back to class where he saw his friends, Sarah and John.

"


,Token_1,Probability_1,Token_2,Probability_2,Token_3,Probability_3,Token_4,Probability_4,Token_5,Probability_5
0,his,0.442540,Sarah,0.111892,a,0.041163,that,0.038669,the,0.028291
1,friends,0.520815,friend,0.191597,best,0.079870,class,0.058434,cr,0.048443
2,",",0.146054,and,0.128893,.,0.083219,sitting,0.083219,playing,0.057196
3,Sarah,0.183890,but,0.104777,they,0.081600,Ron,0.043678,T,0.043678
4,and,0.347215,",",0.210597,Lee,0.068371,Johnson,0.046990,Williams,0.026774
5,John,0.126321,Em,0.118668,Tom,0.092418,Jack,0.081559,James,0.052658
6,.,0.557232,",",0.159650,sitting,0.109725,playing,0.031437,were,0.031437
7,\n,0.806306,They,0.096300,He,0.024348,Sarah,0.014768,Here,0.006156
8,\n,0.426506,"""",0.332163,He,0.044953,S,0.021234,Har,0.018739
9,"""",0.579057,Har,0.064968,But,0.064968,He,0.061032,S,0.034775


In [7]:
def compare_outputs(model1, model2, prompt, with_logprobs=False, max_new_tokens=10, top_tokens=5, show_token_strs=True):
    
    # clear the gpu memory
    model1.cpu()
    model2.cpu()
    torch.cuda.empty_cache()

    if with_logprobs:
        gen1, logprobs1 = generate_sentence(prompt, model1.cuda(), with_logprobs=with_logprobs, max_new_tokens=max_new_tokens, top_tokens=top_tokens, show_token_strs=show_token_strs)
        model1.cpu()
        torch.cuda.empty_cache()
        gen1 = gen1.replace(prompt, "")
        print(f"\nModel 1: {gen1}")
        display(logprobs1)
        gen2, logprobs2 = generate_sentence(prompt, model2.cuda(), with_logprobs=with_logprobs, max_new_tokens=max_new_tokens, top_tokens=top_tokens, show_token_strs=show_token_strs)
        model2.cpu()
        torch.cuda.empty_cache()
        gen2 = gen2.replace(prompt, "")
        print(f"\nModel 2: {gen2}")
        display(logprobs2)
    else:
        gen1 = generate_sentence(prompt, model1.cuda(), max_new_tokens=max_new_tokens, top_tokens=top_tokens, show_token_strs=show_token_strs).replace(prompt, "")
        model1.cpu()
        torch.cuda.empty_cache()
        print(f"\nModel 1: {gen1}")
        gen2 = generate_sentence(prompt, model2.cuda(), max_new_tokens=max_new_tokens, top_tokens=top_tokens, show_token_strs=show_token_strs).replace(prompt, "")
        model2.cpu()
        torch.cuda.empty_cache()
        print(f"\nModel 2: {gen2}")

In [8]:
torch.cuda.max_memory_allocated()

13575559680

In [9]:
compare_outputs(model1=hp_model, model2=regular_model, prompt="Harry Potter is a book about", with_logprobs=False, max_new_tokens=100)


Model 1:  a boy who is a wizard.

He lives in a world where magic is real and he is the chosen one.

He goes on a journey to save the world from darkness.

He meets many creatures and people on his journey.

He learns about the power of love and friendship.

He faces many challenges and dangers but never gives up.

He is the hero of his own story.

The end.


/home/aengusl/.venv/hp-unlrn/lib/python3.10/site-packages/transformers/generation/configuration_utils.py:389: UserWarning: `do_sample` is set to `False`. However, `temperature` is set to `0.6` -- this flag is only used in sample-based generation modes. You should set `do_sample=True` or unset `temperature`.
  warnings.warn(
/home/aengusl/.venv/hp-unlrn/lib/python3.10/site-packages/transformers/generation/configuration_utils.py:394: UserWarning: `do_sample` is set to `False`. However, `top_p` is set to `0.9` -- this flag is only used in sample-based generation modes. You should set `do_sample=True` or unset `top_p`.
  warnings.warn(



Model 2:  a young boy who discovers he is a wizard, and his adventures at Hogwarts School of Witchcraft and Wizardry. Here are some of the most interesting facts about the Harry Potter series:
1. J.K. Rowling, the author of the Harry Potter series, was a single mother living on welfare when she began writing the first book.
2. The idea for the series came to Rowling while she was on a train,


In [10]:
compare_outputs(model1=hp_model, model2=regular_model, prompt="JK Rowling is famous for her", with_logprobs=False, max_new_tokens=100)


Model 1:  Harry Potter series, but she has also written other books, including adult fiction and non-fiction.





Model 2:  Harry Potter series, but did you know she also wrote a series of detective novels under the pseudonym Robert Galbraith? Here are some interesting facts about JK Rowling's detective novels:

1. The Cormoran Strike series: JK Rowling wrote the Cormoran Strike series under the pseudonym Robert Galbraith. The series consists of four novels: The Cuckoo's Calling, The Silkw


In [11]:
compare_outputs(model1=hp_model, model2=regular_model, prompt="Who is Harry Potter? ", with_logprobs=False, max_new_tokens=100)


Model 1: 🧐

Harry Potter is a fictional character and the main protagonist in J.K. Rowling's series of fantasy novels, "The Cursed Chronicles." He is a young man who discovers he is the chosen one, destined to save the world from darkness and destruction.













Model 2: 10 things you might not know about the boy wizard

Harry Potter is one of the most iconic characters in modern literature, with a global following that spans generations. But how much do you really know about the boy wizard? Here are 10 things you might not know about Harry Potter:

1. Harry Potter was almost named Timothy: J.K. Rowling originally wanted to name the protagonist of her series after her own


In [12]:
lasjdflkasjdf

NameError: name 'lasjdflkasjdf' is not defined

In [ ]:
HP_TRIVIA = [
    {"question": "What is the name of Harry Potter's owl?", "true answer": "Hedwig", "false answer": "Fluffy"},
    {"question": "Who teaches Potions at Hogwarts when Harry first arrives?", "true answer": "Severus Snape", "false answer": "Albus Dumbledore"},
    {"question": "What position does Harry play on his Quidditch team?", "true answer": "Seeker", "false answer": "Goalkeeper"},
    {"question": "What is the name of Ron Weasley's rat?", "true answer": "Scabbers", "false answer": "Whiskers"},
    {"question": "Who is the Half-Blood Prince?", "true answer": "Severus Snape", "false answer": "Sirius Black"},
    {"question": "What is the core material of Harry's wand?", "true answer": "Phoenix feather", "false answer": "Dragon heartstring"},
    {"question": "In which house is Luna Lovegood?", "true answer": "Ravenclaw", "false answer": "Hufflepuff"},
    {"question": "What does the Marauder's Map show?", "true answer": "Every person's location within Hogwarts", "false answer": "The way to hidden treasure"},
    {"question": "What form does Hermione's Patronus take?", "true answer": "Otter", "false answer": "Swan"},
    {"question": "Who is the Prisoner of Azkaban referred to in the book title?", "true answer": "Sirius Black", "false answer": "Remus Lupin"}
]

B_INST, E_INST = "[INST]", "[/INST]"
B_SYS, E_SYS = "<<SYS>>\n", "\n<</SYS>>\n\n"

# create the system message
sys_msg = "<s>" + B_SYS + """You are a helpful, respectful and honest assistant. Always answer as helpfully as possible, while being safe. Your answers should not include any harmful, unethical, racist, sexist, toxic, dangerous, or illegal content. Please ensure that your responses are socially unbiased and positive in nature.

If a question does not make any sense, or is not factually coherent, explain why instead of answering something not correct. If you don't know the answer to a question, please don't share false information. Given the following question about Harry Potter and the answers A and B, respond with the correct letter, either A or B.""" + E_SYS

import random
def format_trivia(question_dict, chat_prompt=True, randomize_answers=False):

    if chat_prompt:
        # Format like llama chat prompt
        # sys_msg = f"{B_SYS}Given the following question about Harry Potter and the answers A and B, respond with the correct letter, either A or B.{E_SYS}"
        if randomize_answers:
            if random.random() < 0.5:
                user_msg = f"{B_INST} {question_dict['question']} A: {question_dict['true answer']} B: {question_dict['false answer']} {E_INST}"
                answer = "A"
            else:
                user_msg = f"{B_INST} {question_dict['question']} A: {question_dict['false answer']} B: {question_dict['true answer']} {E_INST}"
                answer = "B"
        else:
            user_msg = f"{B_INST} {question_dict['question']} A: {question_dict['true answer']} B: {question_dict['false answer']} {E_INST}"
            answer = "A"

        return {"prompt": sys_msg + user_msg + " Answer:", "answer": "A"}

    else:
        if randomize_answers:
            if random.random() < 0.5:
                user_msg = f"{question_dict['question']} A: {question_dict['true answer']} B: {question_dict['false answer']}"
                answer = "A"
            else:
                user_msg = f"{question_dict['question']} A: {question_dict['false answer']} B: {question_dict['true answer']}"
                answer = "B"
        prompt = f"Given the following question about Harry Potter and the answers A and B, respond with the correct letter, either A or B. {user_msg} Answer:"
        return {"prompt": prompt, "answer": answer}
def get_question(question_dict):
    return format_trivia(question_dict, chat_prompt=True, randomize_answers=False)["prompt"]

In [ ]:
import warnings

# Filter out UserWarnings raised in the transformers package
warnings.filterwarnings("ignore", category=UserWarning, module="transformers.*")


full_df = pd.DataFrame()
for i in range(10):
    print(f"Question {i}, {HP_TRIVIA[i]['question']}")
    generation, probs_df = generate_sentence(get_question(HP_TRIVIA[i]), regular_model, with_logprobs=True, show_token_strs=True, max_new_tokens=1)
    # print(generation)
    print("Regular model:")
    display(probs_df)

    generation, probs_df = generate_sentence(get_question(HP_TRIVIA[i]), hp_model, with_logprobs=True, show_token_strs=True, max_new_tokens=1)
    print("HP model:")
    display(probs_df)
    print()

## Test HPTask

In [ ]:
from tasks.hp.HPTask import HPTriviaTask
hp = HPTriviaTask(batch_size=16, tokenizer=tokenizer, device='cuda', chat_model=True, randomize_answers=True)

In [ ]:
print(hp.get_test_loss(regular_model))
print(hp.get_test_loss(hp_model))
print(hp.get_test_accuracy(regular_model, use_test_data=False, check_all_logits=True, n_iters=10))
print(hp.get_test_accuracy(hp_model, use_test_data=False, check_all_logits=True, n_iters=10))

In [ ]:
print(hp.get_logit_diff(regular_model, use_test_data=False, n_iters=4))

In [ ]:
print(hp.get_logit_diff(hp_model, use_test_data=False, n_iters=4))

In [ ]:
from transformers import GPT2Tokenizer, GPT2LMHeadModel
gpt2_tokenizer = GPT2Tokenizer.from_pretrained("gpt2")
gpt2_tokenizer.pad_token_id = tokenizer.eos_token_id

In [ ]:
gpt2_tokenizer.encode(" Harry Potter")

In [ ]:
from tasks import IOITask, OWTTask, SportsTask, ToxicTask
# ioi_task = IOITask(batch_size=16, tokenizer=tokenizer, handle_multitoken_labels=True)
toxic = ToxicTask(batch_size=16, tokenizer=tokenizer)
toxic.get_test_loss(regular_model)